In [3]:
import pandas as pd
import seaborn as sns

In [1]:
import os
os.chdir('..')

In [4]:
df_wti = pd.read_csv('data/raw/wti_prices.csv')

In [5]:
df_wti

,Unnamed: 0,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,0,2015-01-02,YCUOK,NaN,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",52.72,$/BBL
1,1,2015-01-05,YCUOK,NaN,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",50.05,$/BBL
2,2,2015-01-06,YCUOK,NaN,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",47.98,$/BBL
3,3,2015-01-07,YCUOK,NaN,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",48.69,$/BBL
4,4,2015-01-08,YCUOK,NaN,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",48.80,$/BBL
...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2703,2025-10-08,YCUOK,NaN,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",63.37,$/BBL
2704,2704,2025-10-09,YCUOK,NaN,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",62.36,$/BBL
2705,2705,2025-10-10,YCUOK,NaN,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",59.75,$/BBL
2706,2706,2025-10-13,YCUOK,NaN,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",59.75,$/BBL
